### setup before running

1. download all 2019 data from "https://webrobots.io/kickstarter-datasets/"
2. make a folder called "raw_data"
3. within "raw_data" make 3 sub-folders with the date of the pulls in the format MM-DD-YYYY
3. extract downloaded data into each of the 3 sub-folders appropriately
4. in line 4 below change the "path_master" variable to your "raw_data" link
5. run (~7min)

### merge all csv

In [ ]:
import re
import os
import pandas as pd
path_master = r'C:/Users/endwy/Documents/Columbia MSBA/Spring 2019/E4650 - Business Analytics/Project - Kickstarter Analysis/raw_data/'
df_kickstarter = pd.DataFrame()

def add_date(path):
    return path[-10:-1]

directories = list()
for root, dirs, files in os.walk(path_master, topdown=False):
    for name in dirs:
        directories.append(os.path.join(root, name)+'/')

file_names = list()
for path, subdirs, files in os.walk(path_master):
    for filename in files:
        if filename not in file_names: file_names.append(filename)

for i in directories: 
    for j in file_names:
        try:
            with open(i+j, encoding='utf8') as fp:
                df_interim = pd.read_csv(fp, keep_default_na=False)
            df_interim['download_date'] = add_date(i)
            df_interim['download_date'] = pd.to_datetime(df_interim['download_date'], format='%m-%d-%Y')
            global df_kickstarter
            df_kickstarter = pd.concat([df_kickstarter,df_interim])
            print('SUCCESS: '+i+j)
        except:
            print('FAIL: '+i+j)
            pass
df_kickstarter.to_csv(path_master+'df_kickstarter'+'.csv')

### read in random sample % of full df_kickstarter to test with

In [2]:
import pandas as pd
import os
import random
filename = r'C:/Users/endwy/Documents/Columbia MSBA/Spring 2019/E4650 - Business Analytics/Project - Kickstarter Analysis/raw_data/df_kickstarter.csv'

p = 0.01  # 1% of the lines
# keep the header, then take only 1% of lines
# if random from [0,1] interval is greater than 0.01 the row will be skipped
df = pd.read_csv(filename, header=0,  skiprows=lambda i: i>0 and random.random() > p)
len(df)
df=df[:1000]
len(df)

1000

### categorize

In [ ]:
cat_json = df['category'].apply(json.loads)
cat_name=list(map(lambda d: d['name'], cat_json))
df['Cat_Name']=cat_name
df.sort_values(['Cat_Name'])

### funding date scraping 

In [3]:
def get_url(x):
    return x.split("\"")[5]

df["urls"] = df["urls"].apply(lambda x: get_url(x))

In [4]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import datetime

def add_fundpd(x,y):
    user_agent_list = [
           #Chrome
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
            'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
            #Firefox
            'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
            'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
            'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
            'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
            'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
        ]
    user_agent = random.choice(user_agent_list)
    heads = {'User-Agent': user_agent}
    #sleep(3)
    try:
        page = requests.get(x,headers=heads)
        results_page = BeautifulSoup(page.content,'lxml')
        period = results_page.find('div',class_='NS_campaigns__funding_period').find_all(class_="js-adjust-time")
        print("success: "+" "+x[36:]+" "+y)
        start = pd.to_datetime(period[0].get_text(), format='%b %d, %Y').strftime('%Y-%m-%d')
        end = pd.to_datetime(period[1].get_text(), format='%b %d, %Y').strftime('%Y-%m-%d')
        delta = int(results_page.find('p',class_='f5').get_text().split("\n")[-2][1:-1].split()[0])
        if y=="start": result = start
        if y=="end": result = end
        if y=="delta": result = delta
        return result
    except:
        print("FAIL: "+x[36:]+" "+y)
        result = "error"
        return result
df["funding_start"] = df["urls"].apply(lambda x: add_fundpd(x,'start'))
df["funding_end"] = df["urls"].apply(lambda x: add_fundpd(x,'end'))
df["funding_delta"] = df["urls"].apply(lambda x: add_fundpd(x,'delta'))

success:  /351383297/choose-my-music-records-6-album-releases-for-2013?ref=category_newest start
success:  /483350489/t-h-e-a-w-a-k-e-n-i-n-g?ref=category_newest start
success:  /the-acquisition-tank/the-acquisition-tank-a-live-multi-discplinary-musi?ref=category_newest start
success:  /1310703504/help-lukas-bryant-fund-terminus-pictures-first-fil?ref=category_newest start
success:  /bebezhang/glow-in-the-dark-daydreamer-enamel-pin?ref=category_newest start
success:  /100221977/ep-suicide-life-de-karasu?ref=category_newest start
success:  /649177195/v-bozsok-music-festival?ref=category_newest start
success:  /296886385/livek-neat-travel-packing-organizers?ref=category_newest start
success:  /1316338235/hels-labyrinth?ref=category_newest start
success:  /johnpender/saint-christopher-loose-canon?ref=category_newest start
success:  /queenboxi/sproutlings-a-compendium-of-little-fictions-sprout?ref=category_newest start
success:  /732999278/blo-the-worlds-first-wearable-cooling-device-and-d

success:  /586244979/jazz-musician-album-and-animated-videos-gravett-dh?ref=category_newest start
success:  /everlastingcreation2/su-weapon-collection?ref=category_newest start
success:  /2066133663/destino-by-michelle-frankfurter-a-photo-book-about?ref=category_newest start
success:  /offendmyeyes/colourful-genderless-clothing-for-creative-colourf?ref=category_newest start
success:  /1700201295/website-design-upgrade-for-pitch-black-sa-pro-socc?ref=category_newest start
success:  /jamesmckeever/combo-and-key-escape-rooms?ref=category_newest start
success:  /1927638179/weapon-brown-aftershock?ref=category_newest start
success:  /1139490928/star-claws-character-egg-cyclopedia?ref=category_newest start
success:  /carlson/angel-clay-and-dragon-clay-lets-finish-the-kiln?ref=category_newest start
success:  /904109796/a-light-of-hope-project-mvemotivation-via-expressi?ref=category_newest start
success:  /myneworleansphoto/2019-mynew-orleans-photo-project-calendar?ref=category_newest start
su

success:  /400279332/vocal-recording-equipment?ref=category_newest start
success:  /538153259/end-o-a-very-funny-short-film-about-endometriosis?ref=category_newest start
success:  /satsmusic/sleeper-and-the-sleepless-new-album?ref=category_newest start
success:  /k1944/kommando-1944-oscar-run-2019?ref=category_newest start
success:  /1335317566/riverwards-produce-market?ref=category_newest start
success:  /ryanhutchens/new-music-from-ryan-hutchens-the-last-ten-years?ref=category_newest start
success:  /bearcarvermusic/bear-carver-full-length-album?ref=category_newest start
success:  /798910678/miss-michellies-big-show?ref=category_newest start
success:  /754854921/our-state-fair-the-film?ref=category_newest start
success:  /1481837183/extra-cheese-please?ref=category_newest start
success:  /2144259853/oh-so-yummy-peach-cobbler?ref=category_newest start
success:  /1055789161/we-go-on-veterans-anthology-for-charity?ref=category_newest start
success:  /928241234/pelo?ref=category_newest s

success:  /bob2015/prettyartcollection?ref=category_newest start
success:  /brandonscottmusic/get-love-stories-off-the-ground-and-into-your-hear?ref=category_newest start
success:  /954048691/ink-minx-a-mobile-tattoo-shop-for-women?ref=category_newest start
success:  /vesuviusmedia/nocturion?ref=category_newest start
success:  /dlimedia/uninvited-guests-a-dandd-5e-and-pathfinder-social?ref=category_newest start
success:  /314900506/lockd-escape-adventures?ref=category_newest start
success:  /tubring/tub-ring-best-of-vinyl-lp-and-new-digital-ep?ref=category_newest start
success:  /1929945916/my-eyes-went-dark?ref=category_newest start
success:  /1707259041/the-brave-little-worm-for-mobile?ref=category_newest start
success:  /1667757478/film-and-photo-for-all-one-year-of-programs-at-the?ref=category_newest start
success:  /756223277/pilgrimage?ref=category_newest start
success:  /1751449133/grease?ref=category_newest start
success:  /573951114/heavens-scent-studios?ref=category_newest st

success:  /1897014110/bambi-lashes-the-ethnicity-lash-that-enhances-eye?ref=category_newest start
success:  /1019896413/glazed-a-columbia-university-thesis-film?ref=category_newest start
success:  /530388301/fundamentals-of-sukokross-and-puzzles?ref=category_newest start
success:  /1651351528/adrian-alicea-spring-summer-2016-a-boy-name-black?ref=category_newest start
success:  /681488087/frankies-world?ref=category_newest start
success:  /rainperry/wanna-be-rain-perrys-icing-on-the-cake?ref=category_newest start
success:  /898094671/sittin-round-waitin-for-the-world-to-end-10-projec?ref=category_newest start
success:  /852214649/mandks-tea-tea-flavored-honey-unique-tea-blends-an?ref=category_newest start
success:  /556520025/original-90s-style-szechuan-nugget-dipping-sauce?ref=category_newest start
success:  /418577699/bad-decisions-make-great-stories-comedy-tour?ref=category_newest start
success:  /952749751/born-in-the-struggle?ref=category_newest start
success:  /1196392781/healthyi

...

FAIL: /1707948331/fairygrocerymother?ref=category_newest delta
success:  /567599608/myaudiosession-worlds-most-personalised-music-expe?ref=category_newest delta
success:  /1852466551/cuddly-and-stretchy-jumbo-anxiety-fox-plush?ref=category_newest delta
success:  /978911160/california-50-states-50-billboards-for-freedoms?ref=category_newest delta
success:  /gunartonline/gao-gun-art-online-project?ref=category_newest delta
success:  /196867341/danielle-ippolitos-d-studio-project?ref=category_newest delta
success:  /312357003/an-ancient-civilization-in-turmoil-forbidden-egypt?ref=category_newest delta
success:  /bloodmnfilm/blood-means-nothing?ref=category_newest delta
success:  /759357419/wearable-power-a-wrist-worn-battery-charger-for-yo?ref=category_newest delta
success:  /stratusgames/casual-game-insider-cgi-year-7?ref=category_newest delta
success:  /mrdiddles/all-natural-surf-wax-mr-diddles-diy-hemp?ref=category_newest delta
FAIL: /172204344/the-arduino-playbot-an-educational-all-in

success:  /nolimitgames/battle-of-souls-trading-card-game?ref=category_newest delta
success:  /766057581/need-craft-supplies-for-starting-up-business?ref=category_newest delta
success:  /801491566/finding-amber-wilde?ref=category_newest delta
success:  /lavoz-cesarvidal/la-voz-de-cesar-vidal-2017-2018-cesarvidalcom?ref=category_newest delta
success:  /1077729994/l8r-creations-custom-wood-working?ref=category_newest delta
success:  /floreskomatsu/in-the-heights-at-the-university-of-michigan?ref=category_newest delta
success:  /imaginarytheaterco/mama-threw-me-so-high-and-he-who-speaks?ref=category_newest delta
success:  /1543718842/by-tessa?ref=category_newest delta
success:  /shawnwrites/the-galaxy-girls-graphic-novel?ref=category_newest delta
success:  /2048228827/deep-dream-prints?ref=category_newest delta
success:  /experiencebash/bash-burger-cos-experience-bash-catering-trailer-b?ref=category_newest delta
success:  /292719246/runners-grinders?ref=category_newest delta
success:  /12

success:  /anaylsis/lefashionplate?ref=category_newest delta
success:  /hunted/imagine-short-film?ref=category_newest delta
success:  /600963220/apistogramma-dwarf-cichlids-database?ref=category_newest delta
success:  /1614220309/marys-son?ref=category_newest delta
success:  /505215049/oasis-pizzaioli?ref=category_newest delta
success:  /1559547824/aura-band-know-your-body-composition-and-hydration?ref=category_newest delta
success:  /1959884182/breaking-ground-animated-short-film?ref=category_newest delta
success:  /1107590672/gem-genie?ref=category_newest delta
success:  /582588669/bisons-sandwich-leisure-food-pork-n-beef-extrodina?ref=category_newest delta
success:  /1400023680/lol-fragespelet-med-nordiga-och-kul-fragor?ref=category_newest delta
success:  /mike-emery/name-doodles-custom-colouring-page-art-commissions?ref=category_newest delta
success:  /1413751810/the-arcadian-wilds-sophomore-record?ref=category_newest delta
success:  /1041243448/good-day-bad-day?ref=category_newest

success:  /1845568511/love-doc?ref=category_newest delta
success:  /229647105/best-dog-gift-ever-never-carry-a-dirty-doggy-bag-a?ref=category_newest delta
success:  /2132690842/playtracking-break-addiction-to-mobile-devices?ref=category_newest delta
success:  /m1engenuity/multiple-enginuity-space-event-theatricsgo?ref=category_newest delta
success:  /1457373989/the-ultimate-kids-headphones?ref=category_newest delta
success:  /413925797/yorktown-lads-first-lp?ref=category_newest delta
success:  /540466948/lux-superhero-horror-comic?ref=category_newest delta
success:  /471834546/smellin-up-the-den?ref=category_newest delta
success:  /84654881/aulaterra-a-classroom-for-ghana?ref=category_newest delta
success:  /2050111439/oil-fundraising-page?ref=category_newest delta
success:  /2019795342/stray-spoon-the-new-batch?ref=category_newest delta
success:  /1659368395/phantasmagoria-show?ref=category_newest delta
success:  /89406624/in-the-dark-0?ref=category_newest delta
success:  /1453963650/

success:  /1271017408/all-the-power-you-cannot-see?ref=category_newest delta
success:  /119565434/new-school-jazz-the-family?ref=category_newest delta
success:  /sarahandthestrange/love-you-lots-like-jelly-tots-a-very-silly-childre?ref=category_newest delta
success:  /390494236/love-still-lives-here-sacred-stories-of-devotion-a?ref=category_newest delta
success:  /1020310203/one-chance-for-one-chance?ref=category_newest delta
success:  /womenofbedford/celebrating-women-in-history-suffragette-statue-be?ref=category_newest delta
success:  /778002582/percko-wear-it-and-get-perfect-posture?ref=category_newest delta
success:  /1884786967/stagelab24?ref=category_newest delta
success:  /1376113826/btb-goes-big?ref=category_newest delta
success:  /1984279110/unrecorded-siege?ref=category_newest delta
success:  /monkeyshinegames/bilder-the-board-game-unleash-your-creativity?ref=category_newest delta
success:  /386876852/informed-voter-app?ref=category_newest delta
FAIL: /605335345/bring-the-tec

In [6]:
df.head(3)

,Unnamed: 0,backers_count,blurb,category,converted_pledged_amount,country,created_at,creator,currency,currency_symbol,...,state,state_changed_at,static_usd_rate,urls,usd_pledged,usd_type,download_date,funding_start,funding_end,funding_delta
0,277,96,Choose My Music Records is a UK indie label. ...,"{""id"":40,""name"":""Indie Rock"",""slug"":""music/ind...",4266,GB,1351932368,"{""id"":351383297,""name"":""Dominik Paczko"",""is_re...",GBP,£,...,successful,1362553810,1.579993,https://www.kickstarter.com/projects/351383297...,4450.841661,international,2019-01-17,2013-02-12,2013-03-06,22
1,360,3,Original urban lifestyle sneaker designed with...,"{""id"":266,""name"":""Footwear"",""slug"":""fashion/fo...",270,US,1499178152,"{""id"":483350489,""name"":""Sir Benni Miles"",""is_r...",USD,$,...,failed,1502212923,1.000000,https://www.kickstarter.com/projects/483350489...,270.000000,domestic,2019-01-17,2017-07-05,2017-08-08,33
2,457,8,The Acquisition Tank is an original science-fi...,"{""id"":24,""name"":""Performance Art"",""slug"":""art/...",918,CA,1404746351,"{""id"":1172834748,""name"":""Adam Charbonneau"",""sl...",CAD,$,...,successful,1410152340,0.938523,https://www.kickstarter.com/projects/the-acqui...,938.522990,domestic,2019-01-17,2014-07-11,2014-09-08,58


In [ ]:
urls = [i for i in df["urls"]]
link = urls[0]

### NLP / WordClouds

In [ ]:
def extract_creator_id(json_obj):
    try:
        temp=json.loads(json_obj)
    except:
        return None
    return temp['id']
    
creator_id = kickstarter_df['creator'].apply(extract_creator_id)

In [ ]:
def extract_reward_urls(json_obj):
    temp = json.loads(json_obj)
    return (temp['web']['rewards'])
    
reward_urls=kickstarter_df['urls'].apply(extract_reward_urls)

In [ ]:
def pull_reward_details(url_list):
    
    reward_tier = list()
    #Read the urls and extract details on rewarding tiers
    for i in url_list:
        #print(i)
        response=requests.get(i)
        if response.status_code!=200:
            print("Error pulling data from {}".format(i))
            continue
        reward_desc=bs(response.content,'lxml')
        a=reward_desc.find('ol').find_all('li')
        reward_tier.append(len(a))
    
    return (reward_tier)
        
reward_tier = pull_reward_details(reward_urls)

In [ ]:
a = pd.DataFrame({"Project_Id":kickstarter_df['id'],
                  "Creator_Id":creator_id,
                  "Reward_Tiers":reward_tier,
                  "Rewards_Url":reward_urls})
a

### overfunding